In [1]:
import pulp as p
import numpy as np
import math
import pandas as pd

from scipy import stats

import random

import math 

<h3>Practice Problem 3: RoofMart</h3>
    
Local retailer RoofMart has three trade lanes for which it is now procuring transport services. For each lane they will transport 10 full truck loads per week. Three carriers have submitted bids for the trade lanes. Their prices per truckload and maximum capacities per week (in parenthesis) are shown below.

<img src = 'w7pp11.png'>

Part 1

Build a spreadsheet model that minimizes the purchase cost for RoofMart.

What is the cost of the optimal solution? Answer in integer dollars without commas or currency symbols.


In [6]:
lanes = ['S-B','S-P','S-N']
laneloads = np.array([10,10,10])

suppliers = ['BigLoad', 'ShipFast', 'TruckerCo']
                     #lane1...laneN
supprice = np.array([[120,80,100],#supp1..
                      [110,80,120],
                      [150,60,40]])#supp3

                     #lane1...laneN
supcap = np.array([[10,10,8],#supp1..
                   [8,8,10],
                   [20,1,1]])#supp3

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers, lanes], supcap)

prob = p.LpProblem('Roof_Mart', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    for l in lanes:
        prob += sup_lanes_flow[s][l] <= supcapdict[s][l]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  2860.0
Solution Status: Optimal
Flow_BigLoad_S_B  =  2.0
Flow_BigLoad_S_N  =  8.0
Flow_BigLoad_S_P  =  9.0
Flow_ShipFast_S_B  =  8.0
Flow_ShipFast_S_N  =  1.0
Flow_ShipFast_S_P  =  0.0
Flow_TruckerCo_S_B  =  0.0
Flow_TruckerCo_S_N  =  1.0
Flow_TruckerCo_S_P  =  1.0


<b>Part 2</b>

Suppose RoofMart wants to impose the constraint that not more than 75% of the volume of any trade lane should be handled by a single carrier.

What is the cost of the optimal solution under this constraint? Answer in dollars without commas or currency symbols.

In [8]:
lanes = ['S-B','S-P','S-N']
laneloads = np.array([10,10,10])

suppliers = ['BigLoad', 'ShipFast', 'TruckerCo']
                     #lane1...laneN
supprice = np.array([[120,80,100],#supp1..
                      [110,80,120],
                      [150,60,40]])#supp3

                     #lane1...laneN
supcap = np.array([[10,10,8],#supp1..
                   [8,8,10],
                   [20,1,1]])#supp3

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers, lanes], supcap)

prob = p.LpProblem('Roof_Mart', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    for l in lanes:
        prob += sup_lanes_flow[s][l] <= supcapdict[s][l]
        prob += sup_lanes_flow[s][l] <= 0.75*laneloadsdict[l] # not more than 75% of the volume of any trade lane should be handled by a single carrier.
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  2890.0
Solution Status: Optimal
Flow_BigLoad_S_B  =  3.0
Flow_BigLoad_S_N  =  7.0
Flow_BigLoad_S_P  =  2.0
Flow_ShipFast_S_B  =  7.0
Flow_ShipFast_S_N  =  2.0
Flow_ShipFast_S_P  =  7.0
Flow_TruckerCo_S_B  =  0.0
Flow_TruckerCo_S_N  =  1.0
Flow_TruckerCo_S_P  =  1.0


<h3>Practice Problem 4: YouKea</h3>
    
Remember Youkea, the upcoming Scandinavian furniture retailer?

As you may remember, Youkea has three stores located in Boston, Providence and New Haven, and has, in anticipation of future growth, just opened a small distribution center in Springfield, MA. Youkea plans to ship 10 truckloads per week to each store from the distribution center and now needs help in the procurement of these transports.

The regional supply chain director is currently deciding on which carriers to contract for the shipments from the distribution center to the stores. A few months ago she sent out a request for proposals (RFPs) and now three carriers have submitted proposals: BigLoad, Shipfast, and TruckerCo. BigLoad and Shipfast are large nationwide carriers, whereas TruckerCo is a small family-run company that mainly focuses on transports from the Boston area to Springfield and now sees a good opportunity to balance its loads and grow its business.

The bids (price per truckload) and carrier capacity (truckloads per week) are given in the table below.

<img src = 'w7pp41.jpg'>

Build an optimization model in your spreadsheet software to help Youkea.

Based on the above information, what is the minimum cost for the network (subject to the capacity constraints)? Answer in integer currency units without decimals or commas.

<b>Part 2</b>

After some research, the director finds out that the on-time performance of the carriers differ. The results of her investigations are seen in the table below.

<img src = 'w7pp42.png'>

The director thinks that %1 service is roughly equal to $10 per truckload.

What is the minimum LOS-adjusted cost, if we normalize LOS at 95%?

<b>Part 3</b>

What is the actual cost with the minimum LOS-adjusted solution? Answer in full currency units without decimals or commas.

In [23]:
lanes = ['S-B','S-P','S-N']
laneloads = np.array([10,10,10])

suppliers = ['BigLoad', 'ShipFast', 'TruckerCo']
                     #lane1...laneN
supprice = np.array([[110,70,100],#supp1..
                      [100,80,120],
                      [90,80,120]])#supp3

                     #lane1...laneN
supcap = np.array([[10,10,8],#supp1..
                   [12,12,10],
                   [8,1,1]])#supp3

#constraint of supplier service level

                     #lane1...laneN
supsl = np.array([[0.95,0.95,0.95],#supp1..
                   [0.97,0.96,0.95],
                   [0.98,0.97,0.9]])#supp3

normalizedLOS = 0.95
LOSprice = 10/0.01

suppriceadj = supprice - (supsl - normalizedLOS)*LOSprice

#=======================================

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], suppriceadj) #use the adjusted supplier price

supcapdict = p.makeDict([suppliers, lanes], supcap)

prob = p.LpProblem('Lanes_Provider_Selection', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    for l in lanes:
        prob += sup_lanes_flow[s][l] <= supcapdict[s][l]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    
prob.solve()

print('minimum cost (LOS Adjusted): ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

suppricedict = p.makeDict([suppliers, lanes], supprice)
print('actual minimum cost: ', sum([sup_lanes_flow[s][l].varValue*suppricedict[s][l] for s in suppliers for l in lanes]))

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost (LOS Adjusted):  2370.0
Solution Status: Optimal
actual minimum cost:  2670.0
Flow_BigLoad_S_B  =  0.0
Flow_BigLoad_S_N  =  8.0
Flow_BigLoad_S_P  =  9.0
Flow_ShipFast_S_B  =  2.0
Flow_ShipFast_S_N  =  2.0
Flow_ShipFast_S_P  =  0.0
Flow_TruckerCo_S_B  =  8.0
Flow_TruckerCo_S_N  =  0.0
Flow_TruckerCo_S_P  =  1.0


<b>Part 4</b>

After thinking about the solution and talking to the carriers, the director believes there are strategic reasons to allocate a larger portion of the business with the smaller carrier (TruckerCo). She believes that TruckerCo should be providing transports on each lane.

What is the minimum actual cost for the LOS-adjusted solution after this change? Answer in full currency units without decimals or commas.

In [2]:
lanes = ['S-B','S-P','S-N']
laneloads = np.array([10,10,10])

suppliers = ['BigLoad', 'ShipFast', 'TruckerCo']
                     #lane1...laneN
supprice = np.array([[110,70,100],#supp1..
                      [100,80,120],
                      [90,80,120]])#supp3

                     #lane1...laneN
supcap = np.array([[10,10,8],#supp1..
                   [12,12,10],
                   [8,1,1]])#supp3

#constraint of supplier service level

                     #lane1...laneN
supsl = np.array([[0.95,0.95,0.95],#supp1..
                   [0.97,0.96,0.95],
                   [0.98,0.97,0.9]])#supp3

normalizedLOS = 0.95
LOSprice = 10/0.01

suppriceadj = supprice - (supsl - normalizedLOS)*LOSprice

#=======================================

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], suppriceadj) #use the adjusted supplier price

supcapdict = p.makeDict([suppliers, lanes], supcap)

prob = p.LpProblem('Lanes_Provider_Selection', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    for l in lanes:
        prob += sup_lanes_flow[s][l] <= supcapdict[s][l]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    prob += sup_lanes_flow['TruckerCo'][l] >= 1 #additional constraint TruckerCo must serve each lane
    
prob.solve()

print('minimum cost (LOS Adjusted): ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

suppricedict = p.makeDict([suppliers, lanes], supprice)
print('actual minimum cost: ', sum([sup_lanes_flow[s][l].varValue*suppricedict[s][l] for s in suppliers for l in lanes]))

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost (LOS Adjusted):  2420.0
Solution Status: Optimal
actual minimum cost:  2670.0
Flow_BigLoad_S_B  =  0.0
Flow_BigLoad_S_N  =  8.0
Flow_BigLoad_S_P  =  9.0
Flow_ShipFast_S_B  =  2.0
Flow_ShipFast_S_N  =  1.0
Flow_ShipFast_S_P  =  0.0
Flow_TruckerCo_S_B  =  8.0
Flow_TruckerCo_S_N  =  1.0
Flow_TruckerCo_S_P  =  1.0


<h3>Practice Problem 5: Producer&Gambler</h3>
    
ProducerGambler is a large Fast Moving Consumer Goods company. They are currently in the middle of renegotiating the supply contracts for four related products.

Three suppliers have expressed interest. Two of them has pointed out that there may be significant economies of scope among the products, opting ProducerGambler to go for a combinatorial auction. After preliminary talks with the third supplier, that supplier too has accepted the format and submitted bids.

All bids (expressed as price per "package"), and demand volumes, are found in this Excel-file or this LibreOffice-file. A package can be seen as a product or a combination of products.

ProducerGambler wants to optimize the procurement so that products are purchased at minimum cost.

<img src = 'w7pp51.png'>

<b>Part 1</b>

Build an optimization model in your spreadsheet software for the combinatorial purchase.

What is the cost of the optimal solution? Answer in full currency units without commas or currency symbols.

In [25]:
products = ['A','B','C','D']
productdem = np.array([12,10,14,11])

suppliers = ['Supplier1', 'Supplier2', 'Supplier3']

packages = ['A','B','C','D','AB','AC','AD','BC','BD','CD','ABC','ABD','ACD','BCD','ABCD']

                     #package1...packageN
supprice = np.array([[489,433,464,467,846,712,748,741,758,823,1112,1199,1056,1672,1872],#sup1...
                     [427,446,494,452,708,758,748,712,734,714,1039,1199,1112,1693,1873],
                     [407,495,414,472,829,832,752,803,771,736,1041,1166,1073,1560,1780]])#sup3

productdemdict = p.makeDict([products], productdem)

suppricedict = p.makeDict([suppliers, packages], supprice)

prob = p.LpProblem('Package_Selection', p.LpMinimize)

package_flow = p.LpVariable.dicts("Flow", (suppliers,packages), lowBound = 0, cat = 'Integer')

prob += p.lpSum([package_flow[s][c] * suppricedict[s][c] for s in suppliers for c in packages])

#if for example, an ABC package is chosen, the flow represents all products in the packages
#i.e. 10 ABC package means I buy 10 A, 10 B, and 10 C product using the ABC package price
#constraint below is that the number of products bought should match the demand
for d in products:
    #only sum if product is in package
    prob += p.lpSum([package_flow[s][c] for s in suppliers for c in packages if d in c]) == productdemdict[d]
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  16566.0
Solution Status: Optimal
Flow_Supplier1_A  =  0.0
Flow_Supplier1_AB  =  0.0
Flow_Supplier1_ABC  =  0.0
Flow_Supplier1_ABCD  =  0.0
Flow_Supplier1_ABD  =  0.0
Flow_Supplier1_AC  =  0.0
Flow_Supplier1_ACD  =  5.0
Flow_Supplier1_AD  =  0.0
Flow_Supplier1_B  =  0.0
Flow_Supplier1_BC  =  0.0
Flow_Supplier1_BCD  =  0.0
Flow_Supplier1_BD  =  0.0
Flow_Supplier1_C  =  0.0
Flow_Supplier1_CD  =  0.0
Flow_Supplier1_D  =  0.0
Flow_Supplier2_A  =  0.0
Flow_Supplier2_AB  =  1.0
Flow_Supplier2_ABC  =  6.0
Flow_Supplier2_ABCD  =  0.0
Flow_Supplier2_ABD  =  0.0
Flow_Supplier2_AC  =  0.0
Flow_Supplier2_ACD  =  0.0
Flow_Supplier2_AD  =  0.0
Flow_Supplier2_B  =  0.0
Flow_Supplier2_BC  =  0.0
Flow_Supplier2_BCD  =  0.0
Flow_Supplier2_BD  =  3.0
Flow_Supplier2_C  =  0.0
Flow_Supplier2_CD  =  3.0
Flow_Supplier2_D  =  0.0
Flow_Supplier3_A  =  0.0
Flow_Supplier3_AB  =  0.0
Flow_Supplier3_ABC  =  0.0
Flow_Supplier3_ABCD  =  0.0
Flow_Supplier3_ABD  =  0.0
Flow_Supplier3_AC  =  0.0
Flow_Supp

<b>Part 2</b>

ProducerGambler is worried about having some products supplied solely by one supplier, and wants to explore what a solution with at least two suppliers for every product would look like.

What is the minimum cost if, for every product, there is at least two suppliers? Answer in full currency units without commas or currency symbols.

In [51]:
products = ['A','B','C','D']
productdem = np.array([12,10,14,11])

suppliers = ['Supplier1', 'Supplier2', 'Supplier3']

packages = ['A','B','C','D','AB','AC','AD','BC','BD','CD','ABC','ABD','ACD','BCD','ABCD']

                     #package1...packageN
supprice = np.array([[489,433,464,467,846,712,748,741,758,823,1112,1199,1056,1672,1872],#sup1...
                     [427,446,494,452,708,758,748,712,734,714,1039,1199,1112,1693,1873],
                     [407,495,414,472,829,832,752,803,771,736,1041,1166,1073,1560,1780]])#sup3

productdemdict = p.makeDict([products], productdem)

suppricedict = p.makeDict([suppliers, packages], supprice)

prob = p.LpProblem('Package_Selection', p.LpMinimize)

package_flow = p.LpVariable.dicts("Flow", (suppliers,packages), lowBound = 0, cat = 'Integer')

prob += p.lpSum([package_flow[s][c] * suppricedict[s][c] for s in suppliers for c in packages])

for s in suppliers:
    for d in products:
        #The constraint below is that:
        #for any supplier, for a particular product they can only supply that product by the product's total demand less one
        #that way it enforce that at least one qty of the demand will be served by other supplier
        prob += p.lpSum([package_flow[s][c] for c in packages if d in c]) <= productdemdict[d] - 1

#if for example, an ABC package is chosen, the flow represents all products in the packages
#i.e. 10 ABC package means I buy 10 A, 10 B, and 10 C product using the ABC package price
#constraint below is that the number of products bought should match the demand
for d in products:
    #only sum if product is in package
    prob += p.lpSum([package_flow[s][c] for s in suppliers for c in packages if d in c]) == productdemdict[d]
      
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  16568.0
Solution Status: Optimal
Flow_Supplier1_A  =  0.0
Flow_Supplier1_AB  =  0.0
Flow_Supplier1_ABC  =  0.0
Flow_Supplier1_ABCD  =  0.0
Flow_Supplier1_ABD  =  0.0
Flow_Supplier1_AC  =  0.0
Flow_Supplier1_ACD  =  5.0
Flow_Supplier1_AD  =  0.0
Flow_Supplier1_B  =  0.0
Flow_Supplier1_BC  =  0.0
Flow_Supplier1_BCD  =  0.0
Flow_Supplier1_BD  =  0.0
Flow_Supplier1_C  =  0.0
Flow_Supplier1_CD  =  0.0
Flow_Supplier1_D  =  0.0
Flow_Supplier2_A  =  0.0
Flow_Supplier2_AB  =  1.0
Flow_Supplier2_ABC  =  5.0
Flow_Supplier2_ABCD  =  0.0
Flow_Supplier2_ABD  =  0.0
Flow_Supplier2_AC  =  0.0
Flow_Supplier2_ACD  =  0.0
Flow_Supplier2_AD  =  0.0
Flow_Supplier2_B  =  0.0
Flow_Supplier2_BC  =  0.0
Flow_Supplier2_BCD  =  0.0
Flow_Supplier2_BD  =  3.0
Flow_Supplier2_C  =  0.0
Flow_Supplier2_CD  =  3.0
Flow_Supplier2_D  =  0.0
Flow_Supplier3_A  =  0.0
Flow_Supplier3_AB  =  0.0
Flow_Supplier3_ABC  =  1.0
Flow_Supplier3_ABCD  =  0.0
Flow_Supplier3_ABD  =  0.0
Flow_Supplier3_AC  =  0.0
Flow_Supp

<h3>Graded Assignment 2: Shiny Star Clothing</h3>

The Shiny Star Clothing company has launched a new clothing line (SKU number HS721) for 
export out of the port of Ningbo. You have been tasked with securing the trucking services to 
handle all of the container freight moves (TEUs) from your four plants to the port of Ningbo. 
Your plants are located in Chongqing, Beijing, Nanjing, and Shijiazhuang.

The lane information is as follows:

Origin	Distance to Ningbo (Km)	Expected Weekly Demand (TEUs)

Chongqing	1750	12

Beijing	1400	20

Nanjing	450	26

Shijiazhuang	1300	14

Currently Shiny Star Clothing has no contracts of any sort with trucking companies to the Ningbo location. You have decided to run an auction and to only invite your top 3 carriers (Red, Blue, and Green) to participate.

Suppose that the carriers have submitted the following bids in Yuan Renminbi per TEU per km.

Origin	Red	Blue	Green

Chongqing	7.50	9.00	8.60

Beijing	6.40	8.30	8.00

Nanjing	8.80	8.90	9.20

Shijiazhuang	7.80	9.20	8.40

<b>Part 1</b>

Based on this data, how many TEUs will you award to each carrier in order to minimize total expected costs?

<b>Part 2</b>

You decide to contact your carriers to make sure they have sufficient capacity to handle your business. The carriers reply with the following capacity in terms of number of TEU loads handled per week in total across all four lanes.

Red: 15 TEUs per week

Blue: 25 TEUs per week

Green: 90 TEUs per week

Based on this new data, how many TEUs will you award to each carrier in order to minimize total expected costs?

<b>Part 3</b>

You want to ensure that each lane is awarded to one and only one carrier. That is, only one carrier can be awarded volume on a lane -no split assignments. Keep the carrier capacity constraints introduced in Part 2.

Based on this new data, how many lanes will you award to each carrier in order to minimize total expected costs?

<b>Part 4</b>

Your manager has another idea. She wants to ensure that each lane has at least two carriers assigned to it - with at least 5 loads per carrier (per lane). Keep the carrier capacity constraints introduced in Part 2.

Based on this new data, how many lanes will you award to each carrier in order to minimize total expected costs?

In [33]:
lanes = ['Chongqing','Beijing','Nanjing', 'Shijiazhuang']
laneloads = np.array([12,20,26,14])
lanedist = np.array([1750,1400,450,1300])

suppliers = ['Red', 'Blue', 'Green']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[7.5,6.4,8.8,7.8],#supp1..
                      [9,8.3,8.9,9.2],
                      [8.6,8,9.2,8.4]])#supp3
        
supcap = np.array([15,25,90]) #PART2

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice*lanedist)

supcapdict = p.makeDict([suppliers], supcap) #PART2

prob = p.LpProblem('Lanes_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')
sup_lanes_flow_open = p.LpVariable.dicts("Open_Flow", (suppliers,lanes), lowBound = 0, cat = 'Binary') #PART3 & PART4

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

#PART2 Constraint
for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    #prob += p.lpSum([sup_lanes_flow_open[s][l] for s in suppliers]) == 1 #PART3
    prob += p.lpSum([sup_lanes_flow_open[s][l] for s in suppliers]) >= 2 #PART4
    
#PART3 Constraint
for s in suppliers:
    for l in lanes:
        prob += sup_lanes_flow[s][l] - sum(laneloads)*sup_lanes_flow_open[s][l] <= 0 #PART3 & PART4 linking flow constraint
        prob += sup_lanes_flow_open[s][l]*5 <= sup_lanes_flow[s][l] #PART4 constraint, if a lane is open, at least 5 loads must be carried

    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  635595.0
Solution Status: Optimal
Flow_Blue_Beijing  =  0.0
Flow_Blue_Chongqing  =  0.0
Flow_Blue_Nanjing  =  20.0
Flow_Blue_Shijiazhuang  =  5.0
Flow_Green_Beijing  =  10.0
Flow_Green_Chongqing  =  7.0
Flow_Green_Nanjing  =  6.0
Flow_Green_Shijiazhuang  =  9.0
Flow_Red_Beijing  =  10.0
Flow_Red_Chongqing  =  5.0
Flow_Red_Nanjing  =  0.0
Flow_Red_Shijiazhuang  =  0.0
Open_Flow_Blue_Beijing  =  0.0
Open_Flow_Blue_Chongqing  =  0.0
Open_Flow_Blue_Nanjing  =  1.0
Open_Flow_Blue_Shijiazhuang  =  1.0
Open_Flow_Green_Beijing  =  1.0
Open_Flow_Green_Chongqing  =  1.0
Open_Flow_Green_Nanjing  =  1.0
Open_Flow_Green_Shijiazhuang  =  1.0
Open_Flow_Red_Beijing  =  1.0
Open_Flow_Red_Chongqing  =  1.0
Open_Flow_Red_Nanjing  =  0.0
Open_Flow_Red_Shijiazhuang  =  0.0


<h3>Graded Assignment 3: Optimization Based Procurement</h3>
    
GetFit is a company that makes high-end exercise machines and the machine Pilates Power is their most popular product. For each production cycle of the Pilates Power machines, GetFit purchases 5 SKUs: W32A, Y33A, W32B, X33B, and Z32A. There are four suppliers: Supplier A, B, C, and D. The number of units needed in each production cycle and the prices for each unit offered by the suppliers are listed in Table 1a. The maximum total number of units that each company can supply are listed in Table 1b. For example, if Supplier C supplies 9000 pieces of SKU Z32A, it cannot supply anything else.

Table 1a. Number of units needed and unit prices (in dollars) offered by the suppliers.

<img src = 'w7ga31.png'>

What is the minimum total procurement cost for GetFit?

What is the total number of units purchased from these companies?



In [35]:
lanes = ['W32A','Y33A','W32B','X33B','Z32A']
laneloads = np.array([24000,9000,24000,18000,11000])

suppliers = ['Supplier A','Supplier B','Supplier C','Supplier D']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[500,109,27,230,324],
                     [538,108,28,225,354],
                     [570,116,30,207,352],
                     [566,117,29,245,378]])#suppN
        
supcap = np.array([31500,7200,9000,41300])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap) #PART2

prob = p.LpProblem('Supplier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

#PART2 Constraint
for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]

    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  21412000.0
Solution Status: Optimal
Flow_Supplier_A_W32A  =  24000.0
Flow_Supplier_A_W32B  =  0.0
Flow_Supplier_A_X33B  =  0.0
Flow_Supplier_A_Y33A  =  0.0
Flow_Supplier_A_Z32A  =  7500.0
Flow_Supplier_B_W32A  =  0.0
Flow_Supplier_B_W32B  =  0.0
Flow_Supplier_B_X33B  =  3700.0
Flow_Supplier_B_Y33A  =  0.0
Flow_Supplier_B_Z32A  =  3500.0
Flow_Supplier_C_W32A  =  0.0
Flow_Supplier_C_W32B  =  0.0
Flow_Supplier_C_X33B  =  9000.0
Flow_Supplier_C_Y33A  =  0.0
Flow_Supplier_C_Z32A  =  0.0
Flow_Supplier_D_W32A  =  0.0
Flow_Supplier_D_W32B  =  24000.0
Flow_Supplier_D_X33B  =  5300.0
Flow_Supplier_D_Y33A  =  9000.0
Flow_Supplier_D_Z32A  =  0.0


<b>Part 2</b>

The manager at GetFit notices that the suppliers have different levels of service. The levels of service of the companies are listed in Table 2a. The costs per % level of service depends on the SKU, and are listed in Table 2b. The costs and maximum total number of units that can be supplied by each supplier remain the same as in Part 1.

Please normalize the costs to Supplier A's level of service (92%) to get the answer.

<img src = 'w7ga32.png'>

Considering the levels of service of the suppliers, what is the minimum total procurement cost for GetFit?

In [100]:
lanes = ['W32A','Y33A','W32B','X33B','Z32A']
laneloads = np.array([24000,9000,24000,18000,11000])
laneslcost = np.array([10,2,1,3,8])

suppliers = ['Supplier A','Supplier B','Supplier C','Supplier D']

                     #lane1...laneN
supprice = np.array([[500,109,27,230,324],
                     [538,108,28,225,354],
                     [570,116,30,207,352],
                     [566,117,29,245,378]])#suppN
        
supcap = np.array([31500,7200,9000,41300])
supsl = np.array([0.92,0.96,0.95,0.96])

benchmarksl = 0.92

benchmarkmatrix = (supsl - benchmarksl)*100
pricediscmatrix = benchmarkmatrix.reshape((len(benchmarkmatrix),1)) * laneslcost.reshape((1,len(laneslcost)))

supadjprice = supprice - pricediscmatrix

#print(supprice)
#print(supadjprice)

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supadjprice)

supcapdict = p.makeDict([suppliers], supcap)

supsldict = p.makeDict([suppliers], supsl)

prob = p.LpProblem('Supplier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

#PART2 Constraint
for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]

    
prob.solve()

print('minimum cost LOS adjusted: ', p.value(prob.objective))
suppricedict = p.makeDict([suppliers, lanes], supprice)
print('actual minimum cost: ', sum([sup_lanes_flow[s][l].varValue*suppricedict[s][l] for s in suppliers for l in lanes]))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost LOS adjusted:  20943000.0
actual minimum cost:  21440000.0
Solution Status: Optimal
Flow_Supplier_A_W32A  =  20500.0
Flow_Supplier_A_W32B  =  0.0
Flow_Supplier_A_X33B  =  0.0
Flow_Supplier_A_Y33A  =  0.0
Flow_Supplier_A_Z32A  =  11000.0
Flow_Supplier_B_W32A  =  3500.0
Flow_Supplier_B_W32B  =  0.0
Flow_Supplier_B_X33B  =  3700.0
Flow_Supplier_B_Y33A  =  0.0
Flow_Supplier_B_Z32A  =  0.0
Flow_Supplier_C_W32A  =  0.0
Flow_Supplier_C_W32B  =  0.0
Flow_Supplier_C_X33B  =  9000.0
Flow_Supplier_C_Y33A  =  0.0
Flow_Supplier_C_Z32A  =  0.0
Flow_Supplier_D_W32A  =  0.0
Flow_Supplier_D_W32B  =  24000.0
Flow_Supplier_D_X33B  =  5300.0
Flow_Supplier_D_Y33A  =  9000.0
Flow_Supplier_D_Z32A  =  0.0


<b>Part 3</b>

For simplicity, please only use information from Part 1 for this question, please do not use any information from Part 2.

The CEO of GetFit had a college housemate, Jenny, who is starting a company, Supplier E, that can supply parts to GetFit. The CEO offers to help Jenny, and so Jenny proposes a contract where she would supply exactly 20% of all the units GetFit needs for each production cycle. This means that Supplier E will supply 17200 parts in total. Any combination of parts are possible, such as 17200 units of W32A or a mix of 17198 units of Y33A and 2 units of W32B. The unit prices for each SKU offered by Supplier E are listed in Table 3.

Table 3. Prices per part offered by Supplier E in dollars

Parts	Prices per part offered by Supplier E in dollars ($/unit)

W32A	555

Y33A	120

W32B	35

X33B	255

Z32A	359

Under this condition, what is the minimum total procurement cost for GetFit?

In [103]:
lanes = ['W32A','Y33A','W32B','X33B','Z32A']
laneloads = np.array([24000,9000,24000,18000,11000])

suppliers = ['Supplier A','Supplier B','Supplier C','Supplier D', 'Supplier E']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[500,109,27,230,324],
                     [538,108,28,225,354],
                     [570,116,30,207,352],
                     [566,117,29,245,378],
                     [555,120,35,255,359]])#suppN
        
supcap = np.array([31500,7200,9000,41300, 17200])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap) #PART2

prob = p.LpProblem('Supplier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

#PART2 Constraint
for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]

prob += p.lpSum([sup_lanes_flow['Supplier E'][l] for l in lanes]) == supcapdict['Supplier E']
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]

    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])
revenueE = sum([sup_lanes_flow['Supplier E'][l].varValue * suppricedict['Supplier E'][l] for l in lanes])
print('Supplier E Revenue: ', revenueE)

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  21414700.0
Solution Status: Optimal
Supplier E Revenue:  2501000.0
Flow_Supplier_A_W32A  =  24000.0
Flow_Supplier_A_W32B  =  0.0
Flow_Supplier_A_X33B  =  0.0
Flow_Supplier_A_Y33A  =  0.0
Flow_Supplier_A_Z32A  =  7500.0
Flow_Supplier_B_W32A  =  0.0
Flow_Supplier_B_W32B  =  0.0
Flow_Supplier_B_X33B  =  7200.0
Flow_Supplier_B_Y33A  =  0.0
Flow_Supplier_B_Z32A  =  0.0
Flow_Supplier_C_W32A  =  0.0
Flow_Supplier_C_W32B  =  0.0
Flow_Supplier_C_X33B  =  9000.0
Flow_Supplier_C_Y33A  =  0.0
Flow_Supplier_C_Z32A  =  0.0
Flow_Supplier_D_W32A  =  0.0
Flow_Supplier_D_W32B  =  19300.0
Flow_Supplier_D_X33B  =  1800.0
Flow_Supplier_D_Y33A  =  0.0
Flow_Supplier_D_Z32A  =  0.0
Flow_Supplier_E_W32A  =  0.0
Flow_Supplier_E_W32B  =  4700.0
Flow_Supplier_E_X33B  =  0.0
Flow_Supplier_E_Y33A  =  9000.0
Flow_Supplier_E_Z32A  =  3500.0


<h3>MicroMasters Practice Problem 2: AwayDepot</h3>

The ambitious young procurement manager at Do-It-Yourself (DIY) retailer AwayDepot is in the process of procuring transport services for AwayDepot’s distribution network. The network consists of a number of warehouses and stores across the US East coast. In total, transport services need to be purchased for 10 different lanes.

After sending out the requirements to a number of carriers, 5 carriers have submitted bids for the different lanes. The bids and the expected volume for each lane can be found below

<img src = "w7mmproblem1.png">

The manager has now turned to you to help him minimize the total costs. However, to spread the risks in the network, there must be at least two carriers per lane.

In [31]:
lanes = ['1','2','3','4','5','6','7','8','9','10']
laneloads = np.array([12,14,14,14,13,13,12,13,11,14])

suppliers = ['Carrier1','Carrier2','Carrier3','Carrier4', 'Carrier5']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[486,513,561,466,443,458,452,447,420,595],
                     [593,526,426,585,463,531,455,510,568,524],
                     [575,548,542,519,469,439,582,555,472,514],
                     [530,579,428,405,434,588,473,595,563,472],
                     [505,525,449,474,447,528,407,408,582,567]])#suppN
        
supcap = np.array([9999999,9999999,9999999,9999999, 9999999])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap) #PART2

prob = p.LpProblem('Carrier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

sup_lanes_flow_open = p.LpVariable.dicts("Lane_Open", (suppliers,lanes), lowBound = 0, cat = 'Binary')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    prob += p.lpSum([sup_lanes_flow_open[s][l] for s in suppliers]) >= 2
    
for s in suppliers:
    for l in lanes:
        #linking constraint for flow and flow open
        prob += sup_lanes_flow[s][l] - sum(laneloads)*sup_lanes_flow_open[s][l] <= 0
        
        #need to add constraint below, i.e.: if a flow is opened, then there must be at least 1 flow unit
        #if we do not add the constraint below, the flow and flow open will not link as there is no cost associated
        #with opening a flow, and from the equation above, if flow open is 1, flow could be zero
        prob += sup_lanes_flow_open[s][l]*1 <= sup_lanes_flow[s][l]

    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  57713.0
Solution Status: Optimal
Flow_Carrier1_1  =  11.0
Flow_Carrier1_10  =  0.0
Flow_Carrier1_2  =  13.0
Flow_Carrier1_3  =  0.0
Flow_Carrier1_4  =  1.0
Flow_Carrier1_5  =  1.0
Flow_Carrier1_6  =  1.0
Flow_Carrier1_7  =  1.0
Flow_Carrier1_8  =  1.0
Flow_Carrier1_9  =  10.0
Flow_Carrier2_1  =  0.0
Flow_Carrier2_10  =  0.0
Flow_Carrier2_2  =  0.0
Flow_Carrier2_3  =  13.0
Flow_Carrier2_4  =  0.0
Flow_Carrier2_5  =  0.0
Flow_Carrier2_6  =  0.0
Flow_Carrier2_7  =  0.0
Flow_Carrier2_8  =  0.0
Flow_Carrier2_9  =  0.0
Flow_Carrier3_1  =  0.0
Flow_Carrier3_10  =  1.0
Flow_Carrier3_2  =  0.0
Flow_Carrier3_3  =  0.0
Flow_Carrier3_4  =  0.0
Flow_Carrier3_5  =  0.0
Flow_Carrier3_6  =  12.0
Flow_Carrier3_7  =  0.0
Flow_Carrier3_8  =  0.0
Flow_Carrier3_9  =  1.0
Flow_Carrier4_1  =  0.0
Flow_Carrier4_10  =  13.0
Flow_Carrier4_2  =  0.0
Flow_Carrier4_3  =  1.0
Flow_Carrier4_4  =  13.0
Flow_Carrier4_5  =  12.0
Flow_Carrier4_6  =  0.0
Flow_Carrier4_7  =  0.0
Flow_Carrier4_8  =  0.0
Flow

<b>Part 3</b>

After seeing your proposed solution, the manager thinks that the total number of carriers are too many. The cost of setting up the contracts and managing the relationships of so many carriers will simply be too high. To reduce these costs, the manager decides there should be no more than two carriers in total for the entire distribution network.

What is the minimum cost of setting up a contract and maintaining a relationship under which this proposed change is better than the previous solution? Answer in full currency units without commas or currency symbols.

In [32]:
lanes = ['1','2','3','4','5','6','7','8','9','10']
laneloads = np.array([12,14,14,14,13,13,12,13,11,14])

suppliers = ['Carrier1','Carrier2','Carrier3','Carrier4', 'Carrier5']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[486,513,561,466,443,458,452,447,420,595],
                     [593,526,426,585,463,531,455,510,568,524],
                     [575,548,542,519,469,439,582,555,472,514],
                     [530,579,428,405,434,588,473,595,563,472],
                     [505,525,449,474,447,528,407,408,582,567]])#suppN
        
supcap = np.array([9999999,9999999,9999999,9999999, 9999999])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap) #PART2

prob = p.LpProblem('Carrier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

sup_lanes_flow_open = p.LpVariable.dicts("Lane_Open", (suppliers,lanes), lowBound = 0, cat = 'Binary')

supplier_selected = p.LpVariable.dicts("Supplier_Selected", (suppliers), lowBound = 0, cat = 'Binary')

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
    #linking constraint if there is any flow related to that supplier, that supplier is selected 
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) - sum(laneloads)*supplier_selected[s] <= 0
    
#only max 2 supplier is selected    
prob += p.lpSum([supplier_selected[s] for s in suppliers]) <= 2
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
    prob += p.lpSum([sup_lanes_flow_open[s][l] for s in suppliers]) >= 2
    
for s in suppliers:
    for l in lanes:
        #linking constraint for flow and flow open
        prob += sup_lanes_flow[s][l] - sum(laneloads)*sup_lanes_flow_open[s][l] <= 0
        
        #need to add constraint below, i.e.: if a flow is opened, then there must be at least 1 flow unit
        #if we do not add the constraint below, the flow and flow open will not link as there is no cost associated
        #with opening a flow, and from the equation above, if flow open is 1, flow could be zero
        prob += sup_lanes_flow_open[s][l]*1 <= sup_lanes_flow[s][l]

    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  59613.0
Solution Status: Optimal
Flow_Carrier1_1  =  11.0
Flow_Carrier1_10  =  1.0
Flow_Carrier1_2  =  13.0
Flow_Carrier1_3  =  1.0
Flow_Carrier1_4  =  1.0
Flow_Carrier1_5  =  1.0
Flow_Carrier1_6  =  12.0
Flow_Carrier1_7  =  11.0
Flow_Carrier1_8  =  12.0
Flow_Carrier1_9  =  10.0
Flow_Carrier2_1  =  0.0
Flow_Carrier2_10  =  0.0
Flow_Carrier2_2  =  0.0
Flow_Carrier2_3  =  0.0
Flow_Carrier2_4  =  0.0
Flow_Carrier2_5  =  0.0
Flow_Carrier2_6  =  0.0
Flow_Carrier2_7  =  0.0
Flow_Carrier2_8  =  0.0
Flow_Carrier2_9  =  0.0
Flow_Carrier3_1  =  0.0
Flow_Carrier3_10  =  0.0
Flow_Carrier3_2  =  0.0
Flow_Carrier3_3  =  0.0
Flow_Carrier3_4  =  0.0
Flow_Carrier3_5  =  0.0
Flow_Carrier3_6  =  0.0
Flow_Carrier3_7  =  0.0
Flow_Carrier3_8  =  0.0
Flow_Carrier3_9  =  0.0
Flow_Carrier4_1  =  1.0
Flow_Carrier4_10  =  13.0
Flow_Carrier4_2  =  1.0
Flow_Carrier4_3  =  13.0
Flow_Carrier4_4  =  13.0
Flow_Carrier4_5  =  12.0
Flow_Carrier4_6  =  1.0
Flow_Carrier4_7  =  1.0
Flow_Carrier4_8  =  1.0
Fl

<h3>MicroMasters Practice Problem 3: OptiBid</h3>
    
You are in charge of the transportation procurement for a manufacturing company.  Historically, the firm has only used two carriers to handle its transportation: Delta and Echo.  These are small regional carriers that have grown with your business over time.  They are known as incumbents and your management believes they have valuable knowledge of how your firm operates.  

In your current bid, you are trying to secure carriers for eight (8) traffic lanes: numbered one to eight.  In order to “test the market” your boss has asked you to invite other carriers to the bid in addition to Delta and Echo.  You have five carriers participating in the reverse auction:  

Alpha – a large national carrier with a very large number of trucks

Baker – a mid-sized carrier

Charlie – a mid-sized carrier

Delta – a small regional carrier that is an incumbent

Echo  – a very small local carrier that is also an incumbent. 

The rates per lane and the demand per lane are as follow:

<img src = 'w7mmproblem21.png'>
<img src = 'w7mmproblem22.png'>

<b>Part 1</b>

As a first step, find the lowest cost assignment of carriers to lanes ignoring carrier capacity, incumbency, and maximum number of carriers used in the network.

<b>Part 3</b>

The carriers have provided you with their weekly maximum capacity in terms of the number of loads they can cover. The weekly capacities are:

Alpha 200

Baker 100

Charlie 100

Delta 20

Echo 10

You want to find the lowest cost assignment of carriers to lanes taking carrier capacity into account, but ignoring incumbency as well as the maximum number of carriers used in the network. Run the solver and answer the following questions. Please only enter integer values.

<b>Part 5</b>

Your Vice President of Transportation has requested that you determine what the impact would be if you limited the total number of carriers assigned volume to a maximum of 3.

You want to find the lowest cost assignment of carriers to lanes taking carrier capacity and maximum number of carriers assigned into account, but ignoring incumbency. Re-solve the problem considering this new constraint and answer the following questions. Please enter integer values.

In [42]:
lanes = ['Lane1','Lane2','Lane3','Lane4','Lane5','Lane6','Lane7','Lane8']
laneloads = np.array([10,12,7,18,6,20,11,2])

suppliers = ['Alpha','Baker','Charlie','Delta','Echo']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[1000,1200,1100,950,920,1029,1500,910],
                     [967,1187,1111,913,886,971,1405,911],
                     [993,1204,1110,910,922,1035,1482,938],
                     [966,1186,1114,920,885,970,1429,918],
                     [969,1205,1095,928,940,1028,1400,917]])#suppN
        
supcap = np.array([200,100,100,20,10])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap)

prob = p.LpProblem('Carrier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

supplier_selected = p.LpVariable.dicts("Supplier_Selected", (suppliers), lowBound = 0, cat = 'Binary') #PART5

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) - sum(laneloads)*supplier_selected[s] <= 0 #PART5
    
prob += p.lpSum([supplier_selected[s] for s in suppliers]) <= 3 #PART5
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  89957.0
Solution Status: Optimal
Flow_Alpha_Lane1  =  0.0
Flow_Alpha_Lane2  =  0.0
Flow_Alpha_Lane3  =  0.0
Flow_Alpha_Lane4  =  0.0
Flow_Alpha_Lane5  =  0.0
Flow_Alpha_Lane6  =  0.0
Flow_Alpha_Lane7  =  0.0
Flow_Alpha_Lane8  =  0.0
Flow_Baker_Lane1  =  10.0
Flow_Baker_Lane2  =  12.0
Flow_Baker_Lane3  =  0.0
Flow_Baker_Lane4  =  0.0
Flow_Baker_Lane5  =  6.0
Flow_Baker_Lane6  =  20.0
Flow_Baker_Lane7  =  8.0
Flow_Baker_Lane8  =  2.0
Flow_Charlie_Lane1  =  0.0
Flow_Charlie_Lane2  =  0.0
Flow_Charlie_Lane3  =  0.0
Flow_Charlie_Lane4  =  18.0
Flow_Charlie_Lane5  =  0.0
Flow_Charlie_Lane6  =  0.0
Flow_Charlie_Lane7  =  0.0
Flow_Charlie_Lane8  =  0.0
Flow_Delta_Lane1  =  0.0
Flow_Delta_Lane2  =  0.0
Flow_Delta_Lane3  =  0.0
Flow_Delta_Lane4  =  0.0
Flow_Delta_Lane5  =  0.0
Flow_Delta_Lane6  =  0.0
Flow_Delta_Lane7  =  0.0
Flow_Delta_Lane8  =  0.0
Flow_Echo_Lane1  =  0.0
Flow_Echo_Lane2  =  0.0
Flow_Echo_Lane3  =  7.0
Flow_Echo_Lane4  =  0.0
Flow_Echo_Lane5  =  0.0
Flow_Echo_La

<b>Part 7</b>

You now want to see how the assignment changes if you favor incumbent carriers, Carriers Delta and Echo, by 5%. That is, you will reduce their rates in the objective function by 5% to improve their chances of being awarded any business.

Keeping the carrier capacity constraints and the limit of assigning no more than 3 carriers, find the lowest cost assignment of carriers to lanes while considering carrier incumbency. Re-solve the problem and answer the following questions. Please enter integer values.

In [3]:

lanes = ['Lane1','Lane2','Lane3','Lane4','Lane5','Lane6','Lane7','Lane8']
laneloads = np.array([10,12,7,18,6,20,11,2])

suppliers = ['Alpha','Baker','Charlie','Delta','Echo']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[1000,1200,1100,950,920,1029,1500,910],
                     [967,1187,1111,913,886,971,1405,911],
                     [993,1204,1110,910,922,1035,1482,938],
                     [966,1186,1114,920,885,970,1429,918],
                     [969,1205,1095,928,940,1028,1400,917]]).astype(float)#suppN

incumbentdisc = 0.05 #PART7
supprice[3:5] = supprice[3:5]*(1-incumbentdisc) #give 5% disc to incumbent carrier #PART7
        
supcap = np.array([200,100,100,20,10])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap)

prob = p.LpProblem('Carrier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

supplier_selected = p.LpVariable.dicts("Supplier_Selected", (suppliers), lowBound = 0, cat = 'Binary') #PART5

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])

for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) - sum(laneloads)*supplier_selected[s] <= 0 #PART5
    
prob += p.lpSum([supplier_selected[s] for s in suppliers]) <= 3 #PART5
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
supprice[3:5] = supprice[3:5]/(1-incumbentdisc)
suppricedict = p.makeDict([suppliers, lanes], supprice)
print('actual minimum cost', sum([sup_lanes_flow[s][l].varValue*suppricedict[s][l] for s in suppliers for l in lanes]))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  88246.0
actual minimum cost 90096.0
Solution Status: Optimal
Flow_Alpha_Lane1  =  0.0
Flow_Alpha_Lane2  =  0.0
Flow_Alpha_Lane3  =  0.0
Flow_Alpha_Lane4  =  0.0
Flow_Alpha_Lane5  =  0.0
Flow_Alpha_Lane6  =  0.0
Flow_Alpha_Lane7  =  0.0
Flow_Alpha_Lane8  =  0.0
Flow_Baker_Lane1  =  10.0
Flow_Baker_Lane2  =  0.0
Flow_Baker_Lane3  =  0.0
Flow_Baker_Lane4  =  18.0
Flow_Baker_Lane5  =  6.0
Flow_Baker_Lane6  =  19.0
Flow_Baker_Lane7  =  1.0
Flow_Baker_Lane8  =  2.0
Flow_Charlie_Lane1  =  0.0
Flow_Charlie_Lane2  =  0.0
Flow_Charlie_Lane3  =  0.0
Flow_Charlie_Lane4  =  0.0
Flow_Charlie_Lane5  =  0.0
Flow_Charlie_Lane6  =  0.0
Flow_Charlie_Lane7  =  0.0
Flow_Charlie_Lane8  =  0.0
Flow_Delta_Lane1  =  0.0
Flow_Delta_Lane2  =  12.0
Flow_Delta_Lane3  =  7.0
Flow_Delta_Lane4  =  0.0
Flow_Delta_Lane5  =  0.0
Flow_Delta_Lane6  =  1.0
Flow_Delta_Lane7  =  0.0
Flow_Delta_Lane8  =  0.0
Flow_Echo_Lane1  =  0.0
Flow_Echo_Lane2  =  0.0
Flow_Echo_Lane3  =  0.0
Flow_Echo_Lane4  =  0.0
Flow_Ech

<b>Part 9</b>

As a final run, you want to find the lowest cost assignment of carriers to lanes taking carrier capacity and incumbency into account, but you do NOT want to limit the number of carriers allowed. Keep the 5% incumbency discount and reset the maximum number of carriers allowed to 5.

Rather than placing a hard constraint on the number of carriers allowed, you decide to include the internal cost of $55 per carrier per week to the model if they are assigned one or more loads per week. You determine that this is the additional managerial cost of communicating and coordinating with a carrier.

Re-solve the problem and answer the following questions. Please enter integer values.

<b>Part 10</b>

You think this $55 per carrier per week fixed charge is quite low. What value would it have to be such that the optimal solution is to use just one carrier? Enter your answer in whole dollars.

In [28]:
lanes = ['Lane1','Lane2','Lane3','Lane4','Lane5','Lane6','Lane7','Lane8']
laneloads = np.array([10,12,7,18,6,20,11,2])

suppliers = ['Alpha','Baker','Charlie','Delta','Echo']

#price per km per unit
                     #lane1...laneN
supprice = np.array([[1000,1200,1100,950,920,1029,1500,910],
                     [967,1187,1111,913,886,971,1405,911],
                     [993,1204,1110,910,922,1035,1482,938],
                     [966,1186,1114,920,885,970,1429,918],
                     [969,1205,1095,928,940,1028,1400,917]]).astype(float)#suppN

incumbentdisc = 0.05 #PART7
supprice[3:5] = supprice[3:5]*(1-incumbentdisc) #give 5% disc to incumbent carrier #PART7

fixedcostpercarrier = 1143 #PART9, PART10 (TRIAL AND ERROR)
        
supcap = np.array([200,100,100,20,10])

laneloadsdict = p.makeDict([lanes], laneloads)

suppricedict = p.makeDict([suppliers, lanes], supprice)

supcapdict = p.makeDict([suppliers], supcap)

prob = p.LpProblem('Carrier_Allocation', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (suppliers,lanes), lowBound = 0, cat = 'Integer')

supplier_selected = p.LpVariable.dicts("Supplier_Selected", (suppliers), lowBound = 0, cat = 'Binary') #PART5

prob += p.lpSum([sup_lanes_flow[s][l] * suppricedict[s][l] for s in suppliers for l in lanes])\
        + p.lpSum([supplier_selected[s] * fixedcostpercarrier for s in suppliers]) #PART9

for s in suppliers:
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) <= supcapdict[s]
    prob += p.lpSum([sup_lanes_flow[s][l] for l in lanes]) - sum(laneloads)*supplier_selected[s] <= 0 #PART5
    
prob += p.lpSum([supplier_selected[s] for s in suppliers]) <= 5 #PART9
        
for l in lanes:
    prob += p.lpSum([sup_lanes_flow[s][l] for s in suppliers]) == laneloadsdict[l]
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
supprice[3:5] = supprice[3:5]/(1-incumbentdisc)
suppricedict = p.makeDict([suppliers, lanes], supprice)
print('actual minimum cost', sum([sup_lanes_flow[s][l].varValue*suppricedict[s][l] for s in suppliers for l in lanes]))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  91281.0
actual minimum cost 90138.0
Solution Status: Optimal
Flow_Alpha_Lane1  =  0.0
Flow_Alpha_Lane2  =  0.0
Flow_Alpha_Lane3  =  0.0
Flow_Alpha_Lane4  =  0.0
Flow_Alpha_Lane5  =  0.0
Flow_Alpha_Lane6  =  0.0
Flow_Alpha_Lane7  =  0.0
Flow_Alpha_Lane8  =  0.0
Flow_Baker_Lane1  =  10.0
Flow_Baker_Lane2  =  12.0
Flow_Baker_Lane3  =  7.0
Flow_Baker_Lane4  =  18.0
Flow_Baker_Lane5  =  6.0
Flow_Baker_Lane6  =  20.0
Flow_Baker_Lane7  =  11.0
Flow_Baker_Lane8  =  2.0
Flow_Charlie_Lane1  =  0.0
Flow_Charlie_Lane2  =  0.0
Flow_Charlie_Lane3  =  0.0
Flow_Charlie_Lane4  =  0.0
Flow_Charlie_Lane5  =  0.0
Flow_Charlie_Lane6  =  0.0
Flow_Charlie_Lane7  =  0.0
Flow_Charlie_Lane8  =  0.0
Flow_Delta_Lane1  =  0.0
Flow_Delta_Lane2  =  0.0
Flow_Delta_Lane3  =  0.0
Flow_Delta_Lane4  =  0.0
Flow_Delta_Lane5  =  0.0
Flow_Delta_Lane6  =  0.0
Flow_Delta_Lane7  =  0.0
Flow_Delta_Lane8  =  0.0
Flow_Echo_Lane1  =  0.0
Flow_Echo_Lane2  =  0.0
Flow_Echo_Lane3  =  0.0
Flow_Echo_Lane4  =  0.0
Flow_Ec

<h2>Final Exam Problem 3</h2>

<b>Question 1</b>

OliverJ’s is a Latin American wholesaler of premium flowers that operates in Perú, México, Colombia, and Ecuador. Its current season best sellers are Tulips and Orchids.

You recently heard that Sara Jackson, one of your newest friends from the MicroMasters course, was just hired as a Category manager in charge of Orchids for the entire region. Her first assignment is to run a bidding process to select one or more growers to serve the network orchids needs for the next year.

Getting ready for the process, she obtained from the Logistic manager, Frank, the annual demand for all countries.

<img src = 'finalexam31.png'>
<img src = 'finalexam32.png'>

She decides then to use her SC2x lessons learned to optimize the procurement process.

After Ms. Jackson finished building and running the optimization model, what’s the minimum cost of procuring the annual demand of Orchids for all countries according to this bidding process information?

Note that: You cannot buy half a flower, and you shouldn't be sending any country anything different to what they demand.

In [18]:
types = ['Orchids']
lanes = ['Peru','Mexico','Colombia', 'Ecuador']

laneloads = np.array([[268,211,277,316]])#type1

suppliers = ['Mathillar&Co','Silvestre','Florellina']

supcap = np.array([[390,460,396]])#type1

                     #sup1          #sup2         #sup3
                     #lane1...laneN #lane1...laneN #lane1...laneN
supprice = np.array([[[35,40,25,20],[20,36,48,24],[42,28,35,50]]])#type1
        

laneloadsdict = p.makeDict([types,lanes], laneloads)

suppricedict = p.makeDict([types,suppliers, lanes], supprice)

supcapdict = p.makeDict([types,suppliers], supcap)

prob = p.LpProblem('Orchids&Tulips', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (types,suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[t][s][l] * suppricedict[t][s][l] for t in types for s in suppliers for l in lanes])

for t in types:
    for s in suppliers:
        prob += p.lpSum([sup_lanes_flow[t][s][l] for l in lanes]) <= supcapdict[t][s]
        
    for l in lanes:
        prob += p.lpSum([sup_lanes_flow[t][s][l] for s in suppliers]) == laneloadsdict[t][l]
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  25391.0
Solution Status: Optimal
Flow_Orchids_Florellina_Colombia  =  11.0
Flow_Orchids_Florellina_Ecuador  =  0.0
Flow_Orchids_Florellina_Mexico  =  211.0
Flow_Orchids_Florellina_Peru  =  0.0
Flow_Orchids_Mathillar&Co_Colombia  =  266.0
Flow_Orchids_Mathillar&Co_Ecuador  =  124.0
Flow_Orchids_Mathillar&Co_Mexico  =  0.0
Flow_Orchids_Mathillar&Co_Peru  =  0.0
Flow_Orchids_Silvestre_Colombia  =  0.0
Flow_Orchids_Silvestre_Ecuador  =  192.0
Flow_Orchids_Silvestre_Mexico  =  0.0
Flow_Orchids_Silvestre_Peru  =  268.0


<b>Question 3</b>

Howard Allen, the sustainability manager from the company, informs Sara that some of the growers are located in the same countries they are proposing to serve.

- Matthillar & Co. is located in Colombia

- Silvestre is located in Ecuador

- Florellina is located in Mexico

As the company is exploring implementing new sustainable procurement practices (which in OliverJ’s is called ‘Susty-47 project’), the team is exploring a scenario in which each vendor should supply at least 50% of the demand in its same country. Once that has been served, their remaining capacity can be allocated either to the same or any other country according to the model results.

Adding the new constraint to the model and running it again for an optimized solution, she is informing the team that the new optimized cost is:

In [20]:
types = ['Orchids']
lanes = ['Peru','Mexico','Colombia', 'Ecuador']

laneloads = np.array([[268,211,277,316]])#type1

suppliers = ['Mathillar&Co','Silvestre','Florellina']

supcap = np.array([[390,460,396]])#type1

                     #sup1          #sup2         #sup3
                     #lane1...laneN #lane1...laneN #lane1...laneN
supprice = np.array([[[35,40,25,20],[20,36,48,24],[42,28,35,50]]])#type1
        

laneloadsdict = p.makeDict([types,lanes], laneloads)

suppricedict = p.makeDict([types,suppliers, lanes], supprice)

supcapdict = p.makeDict([types,suppliers], supcap)

prob = p.LpProblem('Orchids&Tulips', p.LpMinimize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (types,suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[t][s][l] * suppricedict[t][s][l] for t in types for s in suppliers for l in lanes])

for t in types:
    for s in suppliers:
        prob += p.lpSum([sup_lanes_flow[t][s][l] for l in lanes]) <= supcapdict[t][s]
        
    for l in lanes:
        prob += p.lpSum([sup_lanes_flow[t][s][l] for s in suppliers]) == laneloadsdict[t][l]

prob += sup_lanes_flow['Orchids']['Mathillar&Co']['Colombia'] >= 0.5*laneloadsdict['Orchids']['Colombia']
prob += sup_lanes_flow['Orchids']['Silvestre']['Ecuador'] >= 0.5*laneloadsdict['Orchids']['Ecuador']
prob += sup_lanes_flow['Orchids']['Florellina']['Mexico'] >= 0.5*laneloadsdict['Orchids']['Mexico']
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  25391.0
Solution Status: Optimal
Flow_Orchids_Florellina_Colombia  =  11.0
Flow_Orchids_Florellina_Ecuador  =  0.0
Flow_Orchids_Florellina_Mexico  =  211.0
Flow_Orchids_Florellina_Peru  =  0.0
Flow_Orchids_Mathillar&Co_Colombia  =  266.0
Flow_Orchids_Mathillar&Co_Ecuador  =  124.0
Flow_Orchids_Mathillar&Co_Mexico  =  0.0
Flow_Orchids_Mathillar&Co_Peru  =  0.0
Flow_Orchids_Silvestre_Colombia  =  0.0
Flow_Orchids_Silvestre_Ecuador  =  192.0
Flow_Orchids_Silvestre_Mexico  =  0.0
Flow_Orchids_Silvestre_Peru  =  268.0


<b>Question 4</b>

Until Howard gets the ‘Susty-47 project’ scenario approved by the CEO, Sara Jackson continues working in the base optimized model created for Questions 1 and 2.

Because of her awesome job, she’s now been asked to incorporate all seasonal flowers to the model.

<img src = 'finalexam33.png'>

She contacts the growers for additional information, and finds out that on Tulips, not all markets are served by all growers:

Note: If no price in the table, it means that the supplier is not serving the country on that product.

<img src = 'finalexam34.png'>

The finance team gave her additional details to consider for this new model:

-Price of Orchids: 75 per unit

-Price of Tulips: 120 per unit

-Profit is determined as Price - Cost.

If Sara aims now to analyze OliverJ’s profit given the new available data, what’s the maximum annual profit the company can obtain on this new model, considering both, Tulips and Orchids sales?

In [21]:
types = ['Orchids','Tulips']
lanes = ['Peru','Mexico','Colombia', 'Ecuador']

                     #lane1...    #laneN
laneloads = np.array([[268,211,277,316], #type1
                      [250,197,258,294]])#type2

suppliers = ['Mathillar&Co','Silvestre','Florellina']
                    
                  #sup1...#supN
supcap = np.array([[390,460,396],
                   [400,450,300]])#type2

                     #sup1          #sup2         #sup3
                     #lane1...laneN #lane1...laneN #lane1...laneN
supprice = np.array([[[35,40,25,20],[20,36,48,24],[42,28,35,50]],#type1
                     [[22,999999,24,16],[17,15,999999,16],[999999,13,33,16]]])#type2

sellprice = np.array([75,120])
sellpricedict = p.makeDict([types], sellprice)

laneloadsdict = p.makeDict([types,lanes], laneloads)

suppricedict = p.makeDict([types,suppliers, lanes], supprice)

supcapdict = p.makeDict([types,suppliers], supcap)

prob = p.LpProblem('Orchids&Tulips', p.LpMaximize)

sup_lanes_flow = p.LpVariable.dicts("Flow", (types,suppliers,lanes), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_lanes_flow[t][s][l]*sellpricedict[t] for t in types for s in suppliers for l in lanes]) \
        - p.lpSum([sup_lanes_flow[t][s][l] * suppricedict[t][s][l] for t in types for s in suppliers for l in lanes])

for t in types:
    for s in suppliers:
        prob += p.lpSum([sup_lanes_flow[t][s][l] for l in lanes]) <= supcapdict[t][s]
        
    for l in lanes:
        prob += p.lpSum([sup_lanes_flow[t][s][l] for s in suppliers]) == laneloadsdict[t][l]
        
prob.solve()

print('maximum profit: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

maximum profit:  157182.0
Solution Status: Optimal
Flow_Orchids_Florellina_Colombia  =  11.0
Flow_Orchids_Florellina_Ecuador  =  0.0
Flow_Orchids_Florellina_Mexico  =  211.0
Flow_Orchids_Florellina_Peru  =  0.0
Flow_Orchids_Mathillar&Co_Colombia  =  266.0
Flow_Orchids_Mathillar&Co_Ecuador  =  124.0
Flow_Orchids_Mathillar&Co_Mexico  =  0.0
Flow_Orchids_Mathillar&Co_Peru  =  0.0
Flow_Orchids_Silvestre_Colombia  =  0.0
Flow_Orchids_Silvestre_Ecuador  =  192.0
Flow_Orchids_Silvestre_Mexico  =  0.0
Flow_Orchids_Silvestre_Peru  =  268.0
Flow_Tulips_Florellina_Colombia  =  0.0
Flow_Tulips_Florellina_Ecuador  =  0.0
Flow_Tulips_Florellina_Mexico  =  197.0
Flow_Tulips_Florellina_Peru  =  0.0
Flow_Tulips_Mathillar&Co_Colombia  =  258.0
Flow_Tulips_Mathillar&Co_Ecuador  =  142.0
Flow_Tulips_Mathillar&Co_Mexico  =  0.0
Flow_Tulips_Mathillar&Co_Peru  =  0.0
Flow_Tulips_Silvestre_Colombia  =  0.0
Flow_Tulips_Silvestre_Ecuador  =  152.0
Flow_Tulips_Silvestre_Mexico  =  0.0
Flow_Tulips_Silvestre_Peru 